In [ ]:
!pip install transformers --quiet
!pip install sentencepiece --quiet
!pip install datasets --quiet
!pip install evaluate --quiet

!pip install git+https://github.com/google-research/bleurt.git -q
#!wget -N https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip . -q
#!unzip -q -n BLEURT-20.zip
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D12.zip . -q
!unzip -q -n BLEURT-20-D12.zip

### 2 Import libraries

In [ ]:
from datasets import load_dataset, load_metric
from transformers import BertTokenizer, TFBertModel, BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel
import evaluate
import numpy as np
import tensorflow as tf
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import math
from csv import writer

from bleurt import score

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load BLEURT score
bleurt_checkpoint = "BLEURT-20-D12"

bleurt_metric = score.BleurtScorer(bleurt_checkpoint)

### 3 Model Evaluation

In [ ]:
# Select origin and target languages
orig = "es"
target = "zh"
max_length = 100
min_length = 50
ngram_size = None
beam = None

In [ ]:
# Dictionary to store model checkpoints
pair_checkpoint = {'en-zh': '/checkpoint-9000',
             'en-es': '/checkpoint-10000',
             'es-en': '/checkpoint-10000',
             'es-zh': '/checkpoint-8500',
             'zh-es': '/checkpoint-8500',
             'zh-en': '/checkpoint-9000'}

In [ ]:
# Data paths
train_file = f'drive/MyDrive/MIDS/W266/Final_Project/bert2bert-finetuned/{orig}_{target}/train_pairs.csv'
val_file = f'drive/MyDrive/MIDS/W266/Final_Project/bert2bert-finetuned/{orig}_{target}/val_pairs.csv'
test_file = f'drive/MyDrive/MIDS/W266/Final_Project/bert2bert-finetuned/{orig}_{target}/test_pairs.csv'

In [ ]:
# Define model path
dir_path = f'drive/MyDrive/MIDS/W266/Final_Project/bert2bert-finetuned/{orig}_{target}'
if not max_length:
  file_path = f'{dir_path}/baseline'
elif not min_length:
  if not ngram_size:
    file_path = f'{dir_path}/max_length_100'
  else:
    file_path = f'{dir_path}/max_length_100/ngram_{ngram_size}'
else:
  if not beam:
    if not ngram_size:
      file_path = f'{dir_path}/max_length_100/min_length_{min_length}'
    else:
      file_path = f'{dir_path}/max_length_100/min_length_{min_length}/ngram_{ngram_size}'
  else:
    file_path = f'{dir_path}/max_length_100/min_length_{min_length}/beam_{beam}'

In [ ]:
file_path

'drive/MyDrive/MIDS/W266/Final_Project/bert2bert-finetuned/es_zh/max_length_100/min_length_50'

### Model instantiation

In [ ]:
# define tokenizer and encoder/decoder
model_checkpoint = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

# Upload saved fine-tuned model
bert2bert_saved = EncoderDecoderModel.from_pretrained(file_path + pair_checkpoint[f'{orig}-{target}'])

In [ ]:
# Load test data
test_df = pd.read_csv(test_file)[[f'{orig}', f'{target}']].head(5000)
test_orig= test_df[f'{orig}'].values.astype(str)
test_labels = test_df[f'{target}'].values.astype(str)

In [ ]:
test_labels

array(['就连医生也无法摆脱这些东西的影响；事实上，在不少例子中，他们实际上创造了偏见。比如，对硬膜外类固醇注射治疗背痛的研究证明，由对此进行日常管理的医生操作更有可能产生积极效果。',
       '欧洲新的黎明',
       '尽管偏执的保守派现在牢牢地控制着伊朗政府，政治精英中的温和派和改革派正在休眠但是并未死亡，他们可能会随着奥巴马的胜利而复活。他们被强硬派赶出了权力舞台，因为强硬派利用这个国家对安全神经过敏的感觉作为操纵选举、压制持不同政见者以及倒转政治和社会改革的借口。数万美军屯兵邻国使这种感觉变得更加强烈。但是当内贾德在2009年6月谋求连任时改革派很有可能对他发起强有力的挑战。',
       ...,
       '在美国人的集体记忆中，后二战时期是个参考点，尽管这多半是段非正常的时期。在二战刚结束的十年里，由于美国市场与战后萧条的欧洲大陆市场的完全隔离，海外竞争实质上对美国经济毫无影响。与此同时，这场战争反而刺激了人们对于汽车、洗衣机、冰箱、割草机、电视机等批量化生产的产品的大量需求，这种需求在战时已经被压抑过久。',
       '俄罗斯也不甘居于人后，最近进行了数十年来规模最大的核演习，以提醒世人它仍是核大国。',
       '此外，皮克提的书还提供了新的视角审视大萧条和第二次世界大战之后的三十来年，将这段时间视为历史的异常期，其出现的原因也许是灾难性事件能够刺激不同寻常的社会凝聚力。在这个经济高速增长的时代，繁荣被广泛分享，所有群体都在进步，但位于低层的人群改善比例最大。'],
      dtype='<U419')

### Model Evaluation

In [ ]:
num_examples = 100
start_index = 0
end_index = num_examples
test_size = len(test_orig)
num_batches = math.ceil(test_size/num_examples)
test_bleurt_scores_file = f'{file_path}/test_bleurt_scores.csv'

In [ ]:
with open(test_bleurt_scores_file, 'a') as f_object:
  # and get a writer object
  writer_object = writer(f_object)
  
  for _ in range(num_batches): 
      # Get predictions
      test_input_ids = tokenizer.batch_encode_plus(test_orig[start_index: end_index], return_tensors="pt", padding=True, truncation=True, max_length=100)
      test_output_token_ids = bert2bert_saved.generate(test_input_ids.input_ids)
      test_decoded = tokenizer.batch_decode(test_output_token_ids, skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=True, max_length = 100)

      # Compute Bleurt scores
      bleurt_scores = bleurt_metric.score(references = test_labels[start_index: end_index], candidates = test_decoded)

      print(bleurt_scores)

      # pass the list as an argument into writerow()
      writer_object.writerow(bleurt_scores)

      # update indices
      start_index = end_index

      if end_index + num_examples > test_size:
        end_index = test_size
      else:
        end_index += num_examples

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 100 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[0.7648698091506958, 0.7427177429199219, 0.2535862326622009, 0.5543982982635498, 0.4549456834793091, 0.6269398927688599, 0.38588112592697144, 0.6504631638526917, 0.6821452975273132, 0.5302323698997498, 0.4917178153991699, 0.39617425203323364, 0.2767411768436432, 0.4506903290748596, 0.5353443622589111, 0.5189030766487122, 0.5024399161338806, 0.7952283620834351, 0.47959989309310913, 0.450026273727417, 0.2831069231033325, 0.5917761921882629, 0.5605936050415039, 0.6330614686012268, 0.37362703680992126, 0.47998249530792236, 0.5286246538162231, 0.3643431067466736, 0.4849244952201843, 0.5122792720794678, 0.5041645169258118, 0.5032040476799011, 0.4888361096382141, 0.4421597719192505, 0.4426354169845581, 0.6456854343414307, 0.6036175489425659, 0.7935951352119446, 0.2786412239074707, 0.6320610642433167, 0.2894963026046753, 0.5116914510726929, 0.6458454728126526, 0.3208610415458679, 0.5693215727806091, 0.4411274790763855, 0.5873630046844482, 0.5489644408226013, 0.7494364976882935, 0.4340555667877

In [ ]:
test_input_ids = tokenizer.batch_encode_plus(test_orig[:5], return_tensors="pt", padding=True, truncation=True, max_length=100)
test_output_token_ids = bert2bert_saved.generate(test_input_ids.input_ids)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 100 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
tokenizer.batch_decode(test_output_token_ids, skip_special_tokens=True, max_length = 100)

['医 生 也 不 愿 意 接 受 这 些 影 响 。 比 如 ， 在 某 些 情 况 下 ， 有 些 人 会 感 觉 到 这 些 影 响 会 导 致 某 些 人 的 心 理 结 果 。 比 如 ， 研 究 表 明 ， 在 某 些 情 况 下 ， 研 究 人 员 在 研 究 中 所 做 的 调 查 中 的 分 析 比 较 有 两 倍 。',
 '欧 洲 的 新 天 堂 之 旅 之 旅 之 谜 之 谜 之 谜 之 谜 之 谜 之 谜 之 谜 之 谜 之 谜 & mdash ; & mdash ; 欧 洲 的 新 天 堂 & mdash ; & mdash ;',
 '尽 管 如 此 ， 保 守 派 保 守 派 在 伊 朗 政 府 控 制 下 的 控 制 权 力 不 管 是 奥 巴 马 还 是 他 的 政 府 都 是 由 奥 巴 马 的 政 治 精 英 们 所 领 导 的 。 他 们 被 迫 在 政 治 上 的 强 硬 态 度 而 且 在 与 邻 国 的 安 全 感 中 ， 他 们 的 政 治 精 神 也 被 迫 在 奥 巴 马',
 '但 是 ， 低 生 产 率 和 失 业 率 的 增 长 是 经 济 增 长 的 基 础 。 尽 管 经 济 增 长 和 失 业 需 要 改 革 ， 但 这 一 观 点 并 不 是 真 正 的 。 尽 管 这 一 观 点 提 供 了 一 个 理 论 ， 但 它 并 不 能 够 建 立 一 个 真 正 的 经 济 复 苏 计 划 。',
 '在 2011 年 11 月 11 日 阿 拉 伯 革 命 （ arab revolution ） 中 ， 一 名 阿 拉 伯 民 众 写 道 阿 拉 伯 革 命 和 稳 定']